In [100]:
from binance import Client
import pandas as pd
import numpy as np

In [101]:
client=Client()
btc=client.get_historical_klines('BTCUSDT', client.KLINE_INTERVAL_5MINUTE, '25 Nov, 2023')

In [102]:
btc[0]

[1700870400000,
 '37713.57000000',
 '37716.78000000',
 '37676.03000000',
 '37705.65000000',
 '42.19892000',
 1700870699999,
 '1590883.97185880',
 3413,
 '20.62566000',
 '777540.97360390',
 '0']

In [103]:
len(btc)

210739

In [104]:
btc_df=pd.DataFrame(btc, columns=['open_timestamp', 'open', 'high', 'low', 'close',
                                  'volume', 'close_timestamp', 'quote_asset_volume', 'n_trades',
                                  'taker_buy_base_asset_vol', 'taker_buy_quote_asset_vol', 'ignore'])

In [105]:
btc_df.head(5)

,open_timestamp,open,high,low,close,volume,close_timestamp,quote_asset_volume,n_trades,taker_buy_base_asset_vol,taker_buy_quote_asset_vol,ignore
0,1700870400000,37713.57000000,37716.78000000,37676.03000000,37705.65000000,42.19892000,1700870699999,1590883.97185880,3413,20.62566000,777540.97360390,0
1,1700870700000,37705.64000000,37715.24000000,37690.08000000,37701.88000000,72.80080000,1700870999999,2744795.17277160,3484,46.46346000,1751743.56712290,0
2,1700871000000,37701.87000000,37717.28000000,37689.49000000,37717.28000000,102.75270000,1700871299999,3873585.27792210,3075,76.38747000,2879540.79668790,0
3,1700871300000,37717.27000000,37738.62000000,37711.54000000,37724.21000000,167.07850000,1700871599999,6303202.69911200,3378,120.46283000,4544564.80168820,0
4,1700871600000,37724.21000000,37724.97000000,37706.99000000,37724.96000000,32.57015000,1700871899999,1228458.38128110,1873,18.54694000,699513.92555500,0


In [106]:
btc_df.drop(columns=['quote_asset_volume', 'taker_buy_base_asset_vol', 'taker_buy_quote_asset_vol', 'ignore'], inplace=True)

In [107]:
btc_df.head(5)

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades
0,1700870400000,37713.57000000,37716.78000000,37676.03000000,37705.65000000,42.19892000,1700870699999,3413
1,1700870700000,37705.64000000,37715.24000000,37690.08000000,37701.88000000,72.80080000,1700870999999,3484
2,1700871000000,37701.87000000,37717.28000000,37689.49000000,37717.28000000,102.75270000,1700871299999,3075
3,1700871300000,37717.27000000,37738.62000000,37711.54000000,37724.21000000,167.07850000,1700871599999,3378
4,1700871600000,37724.21000000,37724.97000000,37706.99000000,37724.96000000,32.57015000,1700871899999,1873


In [108]:
btc_df['open']=pd.to_numeric(btc_df['open'])
btc_df['close']=pd.to_numeric(btc_df['close'])
btc_df['low']=pd.to_numeric(btc_df['low'])
btc_df['high']=pd.to_numeric(btc_df['high'])
btc_df['volume']=pd.to_numeric(btc_df['volume'])

In [109]:
btc_df.head(5)

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades
0,1700870400000,37713.57,37716.78,37676.03,37705.65,42.19892,1700870699999,3413
1,1700870700000,37705.64,37715.24,37690.08,37701.88,72.80080,1700870999999,3484
2,1700871000000,37701.87,37717.28,37689.49,37717.28,102.75270,1700871299999,3075
3,1700871300000,37717.27,37738.62,37711.54,37724.21,167.07850,1700871599999,3378
4,1700871600000,37724.21,37724.97,37706.99,37724.96,32.57015,1700871899999,1873


In [110]:
btc_df['ema_10']=btc_df['close'].ewm(span=10, min_periods=10).mean()
btc_df['ema_30']=btc_df['close'].ewm(span=20, min_periods=20).mean()
btc_df['ema_50']=btc_df['close'].ewm(span=50, min_periods=50).mean()
btc_df['ema_100']=btc_df['close'].ewm(span=100, min_periods=100).mean()
btc_df['ema_200']=btc_df['close'].ewm(span=200, min_periods=200).mean()

In [111]:
btc_df.iloc[200]

open_timestamp     1.700930e+12
open               3.775000e+04
high               3.775000e+04
low                3.772823e+04
close              3.772897e+04
volume             2.061826e+01
close_timestamp    1.700931e+12
n_trades           1.583000e+03
ema_10             3.772921e+04
ema_30             3.771647e+04
ema_50             3.770422e+04
ema_100            3.770936e+04
ema_200            3.772127e+04
Name: 200, dtype: float64

In [112]:
btc_df.dropna(inplace=True)

In [113]:
len(btc_df)

210540

In [114]:
btc_df.head(5)

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,ema_50,ema_100,ema_200
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,37703.213714,37708.953567,37721.184035
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,37704.224090,37709.357177,37721.273494
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,37705.431362,37709.874050,37721.430965
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,37706.837609,37710.506864,37721.658328
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,37708.099632,37711.080988,37721.856786


In [115]:
btc_df.dropna(inplace=True)

In [119]:
def rsi(df: pd.DataFrame, period: int=14):
    price_diff=df['close'].diff()
    gain=(price_diff.where(price_diff>0, 0)).rolling(window=period).mean()
    loss=(-price_diff.where(price_diff<0, 0)).rolling(window=period).mean()

    rs=gain/loss
    rsi=100-(100/(1+rs))

    return rsi

btc_df['rsi']=rsi(btc_df, period=14)

In [121]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,ema_50,ema_100,ema_200,rsi
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,37703.213714,37708.953567,37721.184035,NaN
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,37704.224090,37709.357177,37721.273494,NaN
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,37705.431362,37709.874050,37721.430965,NaN
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,37706.837609,37710.506864,37721.658328,NaN
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,37708.099632,37711.080988,37721.856786,NaN


In [122]:
def macd(df: pd.DataFrame):
    short_ema=df['close'].ewm(span=12, adjust=False).mean()
    long_ema=df['close'].ewm(span=26, adjust=False).mean()

    macd_line=short_ema-long_ema
    signal_line=macd_line.ewm(span=9, adjust=False).mean()

    return macd_line, signal_line

btc_df['macd_line'], btc_df['signal_line']=macd(btc_df)

In [125]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,ema_50,ema_100,ema_200,rsi,macd_line,signal_line
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,37703.213714,37708.953567,37721.184035,NaN,0.000000,0.000000
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,37704.224090,37709.357177,37721.273494,NaN,-1.676809,-0.335362
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,37705.431362,37709.874050,37721.430965,NaN,-2.490414,-0.766372
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,37706.837609,37710.506864,37721.658328,NaN,-2.598504,-1.132799
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,37708.099632,37711.080988,37721.856786,NaN,-2.834660,-1.473171


In [126]:
def bollinger_bands(df: pd.DataFrame, period: int=20):
    df['sma']=df['close'].rolling(window=period).mean()
    df['upper_band']=df['sma']+(df['close'].rolling(window=period).std()*2)
    df['lower_band']=df['sma']-(df['close'].rolling(window=period).std()*2)

bollinger_bands(btc_df)

In [129]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,ema_50,ema_100,ema_200,rsi,macd_line,signal_line,sma,upper_band,lower_band
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,37703.213714,37708.953567,37721.184035,NaN,0.000000,0.000000,NaN,NaN,NaN
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,37704.224090,37709.357177,37721.273494,NaN,-1.676809,-0.335362,NaN,NaN,NaN
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,37705.431362,37709.874050,37721.430965,NaN,-2.490414,-0.766372,NaN,NaN,NaN
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,37706.837609,37710.506864,37721.658328,NaN,-2.598504,-1.132799,NaN,NaN,NaN
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,37708.099632,37711.080988,37721.856786,NaN,-2.834660,-1.473171,NaN,NaN,NaN


In [130]:
def stochastic_oscillator(df: pd.DataFrame, k_period=14, d_period=3):
    high=df['high'].rolling(window=k_period).max()
    low=df['low'].rolling(window=k_period).min()
    k=100*((df['close']-low)/(high-low))
    d=k.rolling(window=d_period).mean()

    return k, d

btc_df['k'], btc_df['d']=stochastic_oscillator(btc_df)

In [132]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,ema_100,ema_200,rsi,macd_line,signal_line,sma,upper_band,lower_band,k,d
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,...,37708.953567,37721.184035,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,...,37709.357177,37721.273494,NaN,-1.676809,-0.335362,NaN,NaN,NaN,NaN,NaN
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,...,37709.874050,37721.430965,NaN,-2.490414,-0.766372,NaN,NaN,NaN,NaN,NaN
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,...,37710.506864,37721.658328,NaN,-2.598504,-1.132799,NaN,NaN,NaN,NaN,NaN
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,...,37711.080988,37721.856786,NaN,-2.834660,-1.473171,NaN,NaN,NaN,NaN,NaN


In [133]:
def wwma(values: pd.Series, alpha):
    return values.ewm(alpha=1/alpha, min_periods=alpha, adjust=False).mean()

def atr(df: pd.DataFrame, period=14):
    data=df.copy()
    high=data['high']
    low=data['low']
    close=data['close']
    data['tr_0']=abs(high-low)
    data['tr_1']=abs(high-close.shift())
    data['tr_2']=abs(low-close.shift())
    tr=data[['tr_0', 'tr_1', 'tr_2']].max(axis=1)
    atr=wwma(tr, period)

    return atr

btc_df['atr']=atr(btc_df)

In [135]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,ema_200,rsi,macd_line,signal_line,sma,upper_band,lower_band,k,d,atr
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,...,37721.184035,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,...,37721.273494,NaN,-1.676809,-0.335362,NaN,NaN,NaN,NaN,NaN,NaN
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,...,37721.430965,NaN,-2.490414,-0.766372,NaN,NaN,NaN,NaN,NaN,NaN
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,...,37721.658328,NaN,-2.598504,-1.132799,NaN,NaN,NaN,NaN,NaN,NaN
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,...,37721.856786,NaN,-2.834660,-1.473171,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
def chaikin_money_flow(df: pd.DataFrame, period=20):
    data=btc_df.copy()
    data['money_flow_multiplier']=((data['close']-data['low'])-(data['high']-data['close']))/(data['high']-data['low'])
    data['money_flow_volume']=data['money_flow_multiplier']*data['volume']
    cmf=data['money_flow_volume'].rolling(window=period).sum()/data['volume'].rolling(window=period).sum()
    
    return cmf

btc_df['chaikin_money_flow']=chaikin_money_flow(btc_df)

In [138]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,rsi,macd_line,signal_line,sma,upper_band,lower_band,k,d,atr,chaikin_money_flow
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,...,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,...,NaN,-1.676809,-0.335362,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,...,NaN,-2.490414,-0.766372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,...,NaN,-2.598504,-1.132799,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,...,NaN,-2.834660,-1.473171,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
# hour of the day
btc_df['open_time']=pd.to_datetime(btc_df['open_timestamp'], unit='ms')
btc_df['close_time']=pd.to_datetime(btc_df['close_timestamp'], unit='ms')

In [140]:
btc_df.head()

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,signal_line,sma,upper_band,lower_band,k,d,atr,chaikin_money_flow,open_time,close_time
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:35:00,2023-11-25 16:39:59.999
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,...,-0.335362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:40:00,2023-11-25 16:44:59.999
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,...,-0.766372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:45:00,2023-11-25 16:49:59.999
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,...,-1.132799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:50:00,2023-11-25 16:54:59.999
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,...,-1.473171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:55:00,2023-11-25 16:59:59.999


In [142]:
btc_df['hour_open']=btc_df['open_time'].dt.hour
btc_df['hour_close']=btc_df['close_time'].dt.hour

In [143]:
btc_df

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,upper_band,lower_band,k,d,atr,chaikin_money_flow,open_time,close_time,hour_open,hour_close
199,1700930100000,37729.86,37754.57,37729.85,37749.99,34.47194,1700930399999,1791,37729.258143,37715.150695,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:35:00,2023-11-25 16:39:59.999,16,16
200,1700930400000,37750.00,37750.00,37728.23,37728.97,20.61826,1700930699999,1583,37729.205753,37716.466819,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:40:00,2023-11-25 16:44:59.999,16,16
201,1700930700000,37728.98,37735.01,37728.84,37735.00,23.53178,1700930999999,1479,37730.259253,37718.231884,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:45:00,2023-11-25 16:49:59.999,16,16
202,1700931000000,37735.01,37747.58,37735.00,37741.28,29.01766,1700931299999,1785,37732.263025,37720.426943,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:50:00,2023-11-25 16:54:59.999,16,16
203,1700931300000,37741.28,37741.29,37734.72,37739.01,16.96482,1700931599999,1781,37733.489748,37722.196758,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-25 16:55:00,2023-11-25 16:59:59.999,16,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210734,1764090600000,87170.80,87510.00,87157.08,87466.82,134.28648,1764090899999,25542,87152.612831,87075.346554,...,87364.617081,86784.608919,94.586598,77.163245,220.807801,0.179601,2025-11-25 17:10:00,2025-11-25 17:14:59.999,17,17
210735,1764090900000,87466.82,87472.87,87229.25,87317.38,50.59463,1764091199999,23136,87182.570498,87098.397358,...,87397.573178,86780.511822,74.612505,82.808872,222.437244,0.111636,2025-11-25 17:15:00,2025-11-25 17:19:59.999,17,17
210736,1764091200000,87317.38,87558.97,87317.38,87512.76,60.49950,1764091499999,14095,87242.604953,87137.860467,...,87467.104421,86745.089579,94.139059,87.779388,223.805298,0.075840,2025-11-25 17:20:00,2025-11-25 17:24:59.999,17,17
210737,1764091500000,87512.76,87625.01,87464.76,87493.44,100.84229,1764091799999,18139,87288.211325,87171.725185,...,87527.171647,86733.142353,80.387276,83.046280,219.265634,0.090874,2025-11-25 17:25:00,2025-11-25 17:29:59.999,17,17


In [177]:
len(btc_df[btc_df['open_time'].dt.weekday==6])

30240

In [151]:
for i in btc_df['open_time']:
    if i.weekday==5 or i.weekday==6:
        btc_df['weekend']=True
    else:
        btc_df['weekend']=False


In [162]:
btc_df[btc_df['weekend']==True]

,open_timestamp,open,high,low,close,volume,close_timestamp,n_trades,ema_10,ema_30,...,lower_band,k,d,atr,chaikin_money_flow,open_time,close_time,hour_open,hour_close,weekend
